In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
books = pd.read_csv('BX-CSV-Dump/BX-Books.csv', sep=';', error_bad_lines=False)
users = pd.read_csv('BX-CSV-Dump/BX-Users.csv', sep=';', error_bad_lines=False)
ratings = pd.read_csv('BX-CSV-Dump/BX-Book-Ratings.csv', sep=';', error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/home/lacho/.local/lib/python3.8/site-packages/IPython/core/interactiveshe

In [ ]:
books.head()

In [ ]:
users.head()

In [3]:
users=users.fillna(0)
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",0.0
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",0.0
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",0.0


In [ ]:
ratings.head()

In [ ]:
ratings['Book-Rating'].describe()

In [ ]:
print(books.shape)
print(users.shape)
print(ratings.shape)

In [ ]:
plt.rc("font", size = 15)
ratings['Book-Rating'].value_counts(sort = False).plot(kind = 'bar')
plt.title('Rating Distribution\n')
plt.xticks(rotation=0, horizontalalignment="center")
plt.xlabel('Rating')
plt.ylabel('Count')

In [ ]:
plt.rc("font", size = 15)
users['Age'].hist(bins=range(10, 110))
plt.title('Age distribution\n')
plt.xlabel('Age')
plt.ylabel('Count')

In [4]:
book_with_rating = pd.merge(ratings, books, on='ISBN')
columns = ['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
book_with_rating = book_with_rating.drop(columns, axis=1)
book_with_rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,2313,034545104X,5
2,6543,034545104X,0
3,8680,034545104X,5
4,10314,034545104X,9


In [5]:
book_with_rating = book_with_rating.dropna(axis = 0, subset = ['ISBN'])
book_rating_count = (book_with_rating.groupby(by = ['ISBN'])['Book-Rating']
                     .count()
                     .reset_index()
                     .rename(columns = {'Book-Rating': 'Rating_Count'}))

book_rating_count.head()

,ISBN,Rating_Count
0,0000913154,1
1,0001010565,2
2,0001046438,1
3,0001046713,1
4,000104687X,1


In [6]:
print(book_rating_count['Rating_Count'].describe())

count    270151.000000
mean          3.816888
std          13.811690
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max        2502.000000
Name: Rating_Count, dtype: float64


In [7]:
print(book_rating_count['Rating_Count'].quantile(np.arange(.9, 1, .01)))

0.90     7.0
0.91     7.0
0.92     8.0
0.93     9.0
0.94    10.0
0.95    12.0
0.96    14.0
0.97    18.0
0.98    25.0
0.99    42.0
Name: Rating_Count, dtype: float64


In [8]:
popular_books = book_rating_count.query('Rating_Count >= 18')
popular_books.head()

,ISBN,Rating_Count
370,0002558122,19
803,000649840X,89
880,0006547834,39
894,0006550576,24
896,0006550789,30


In [9]:
popular_books.shape

(8283, 2)

In [10]:
popular_books_rating = popular_books.merge(book_with_rating, on = 'ISBN')
popular_books_rating.head()
popular_books_rating.shape

(404478, 4)

In [11]:
combined = popular_books_rating.merge(users, on = 'User-ID', how = 'left')
combined.head()

,ISBN,Rating_Count,User-ID,Book-Rating,Location,Age
0,0002558122,19,278474,0,"croydon, new south wales, australia",40.0
1,0002558122,19,11676,8,"n/a, n/a, n/a",0.0
2,0002558122,19,27782,0,"dandenong north, victoria, australia",53.0
3,0002558122,19,59616,0,"port melbourne, victoria, australia",0.0
4,0002558122,19,71837,10,"middle beach, south australia, australia",60.0


In [ ]:
germans_between_age=combined.loc[
    (combined['Age'] >= 18) 
    & (combined['Age'] < 40) 
    & (combined['Location'].str.contains("germany"))
]

germans_between_age=germans_between_age.drop('Location', axis = 1)
germans_between_age = germans_between_age.drop_duplicates(['User-ID', 'ISBN'])
germans_between_age.head()

In [ ]:
germans_between_age.shape

In [ ]:
reviews_matrix = germans_between_age.pivot(index = 'User-ID', columns = 'ISBN', values = 'Book-Rating').fillna(0)
reviews_matrix.head()

In [ ]:
reviews_matrix.shape